# Legislative Mismatch

### Friendly Cities Lab @ Georgia Tech

Analyzes discrepancies between U.S. Congress Representatives’ voting records and their districts’ Social Vulnerability Index.


## Working with Agriculture Data

In [6]:
import chardet

file_path = "./data/cdp.csv"
with open(file_path, "rb") as f:
  result = chardet.detect(f.read())
  encoding = result['encoding']
  print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


Reshape the agriculture data so that each statistic is a column and each row is a congressional district in a state.

In [15]:
import pandas as pd

file_path = "./data/cdp.csv"
df = pd.read_csv(file_path, encoding=encoding)

df = df.drop_duplicates(subset=["State", "Congressional District", "Label"], keep=False)
reshaped_df = df.pivot(index=["State", "Congressional District"], columns="Label", values="Value").reset_index()
reshaped_df.to_csv("./data/reshaped_agriculture_data.csv", index=False)

num_reshaped_columns = len(reshaped_df.columns)
print(f"Reshaped data saved to reshaped_agriculture_data.csv with {num_reshaped_columns} columns.")

Reshaped data saved to reshaped_agriculture_data.csv with 206 columns.


Filter out empty rows and columns to get only the statistics that exist for every congressional district, of which there are 27.

In [57]:
# Drop empty rows (i.e. State no. 36 districts 4-8)
filtered_df = reshaped_df.dropna(subset=[col for col in reshaped_df.columns if col not in ["State", "Congressional District"]], how="all")
# Drop columns with empty rows, ensuring that each remaining column has data for each congressional district
filtered_df = filtered_df.dropna(axis=1, how="any")
filtered_df.to_csv("./data/filtered_agriculture_data.csv", index=False)

num_filtered_columns = len(filtered_df.columns)
print(f"Reshaped data saved to filtered_agriculture_data.csv with {num_filtered_columns}")

Reshaped data saved to filtered_agriculture_data.csv with 27


Rank the variance of each statistic from greatest to least to help determine which statistic to map out.

In [58]:
# Get just the columns holding the statistics
statistical_columns = (
  filtered_df.drop(columns=["State", "Congressional District"])
  .replace(r"[^0-9.\-]", "", regex=True)
  .apply(pd.to_numeric, errors="coerce")
)

variances = statistical_columns.var(skipna=True)
ranked_variances = variances.sort_values(ascending=False).apply(lambda x: f"{x:.3g}")

print("Column variances ranked from greatest to least:")
print(ranked_variances)

Column variances ranked from greatest to least:
Label
Total ($): Market value of products sold                           7.14e+18
Total ($): Total farm production expenses                          4.19e+18
Total ($): Net cash farm income                                    6.54e+17
Total and Per Farm Overview, 2022: Land in farms (acres)           3.12e+13
Market Value of Agricultural Products Sold: Total                  7.14e+12
Market Value of Agricultural Products Sold: Crops                   2.3e+12
Per farm average ($): Market value of products sold                1.77e+11
Per farm average ($): Total farm production expenses                  1e+11
Per farm average ($): Net cash farm income                         1.77e+10
Total Producers                                                    1.06e+08
Race: White                                                        9.96e+07
Sex: Male                                                          4.57e+07
Total and Per Farm Overview, 2022:

Create District ID and convert all statistic columns to numbers to join with congressional district map data.

In [63]:
join_df = filtered_df.copy()

# Pad state and cong district IDs with 0s
join_df['State'] = join_df['State'].astype(str).str.zfill(2)
join_df['Congressional District'] = join_df['Congressional District'].astype(str).str.zfill(2)

# Convert every statistical column to int to display visually on the map
join_df = join_df.replace(r"[^0-9.\-]", "", regex=True).apply(
    lambda col: pd.to_numeric(col, errors='coerce') if col.name not in ['State', 'Congressional District'] else col
)

# Add new column for district id
join_df['District_ID'] = join_df.apply(
  lambda row: row['State'] + '00'
  if (join_df['State'] == row['State']).sum() == 1 else row['State'] + row['Congressional District'],
  axis=1
)

join_df.to_csv("./data/join_agriculture_data.csv", index=False)
print(f"Data ready to join with map.")

Data ready to join with map.
